# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.6661160310000014
GPU (s):
0.05489341000000536
GPU speedup over CPU: 48x


In [ ]:
from google.colab import drive #For using the google drive for data
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np#Array handling
import pandas as pd#Working with dataframes

In [ ]:
train=pd.read_csv('/content/drive/My Drive/dataset/train.csv')
test=pd.read_csv('/content/drive/My Drive/dataset/test.csv')

In [ ]:
train.head() #Check above mapping worked or not

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [ ]:
class_map={'manipuri':0,'bharatanatyam':1,'odissi':2, 'kathakali':3, 'kathak':4,'sattriya':5, 'kuchipudi':6, 'mohiniyattam':7}
inverse_map={0:'manipuri',1:'bharatanatyam',2:'odissi',3:'kathakali',4: 'kathak',5:'sattriya',6:'kuchipudi',7:'mohiniyattam'}
train['target']=train['target'].map(class_map) #maps the two series of class_map[dance forms,ints] dance_forms->ints

In [ ]:
train.head() #Check above mapping worked or not

,Image,target
0,96.jpg,0
1,163.jpg,1
2,450.jpg,2
3,219.jpg,3
4,455.jpg,2


In [ ]:
img_h,img_w=(224,224) #For using the standard models to make use of transfer learning

In [ ]:
import os#This module provides a portable way of using operating system dependent functionality.
import seaborn as sns #For graphs
import cv2 #it is used for computer vision applications like image processing,video capture analysis and like face and object detection
from tqdm import tqdm #Shows the progess bar of how much a loop is executed or a pipeline is executed.
import matplotlib.pyplot as plt #For plotting.

In [ ]:
train_img=[] #In this list we will have images of required type as we needed
train_label=[] #lables for images 
j=0
path='/content/drive/My Drive/dataset/train'
for i in tqdm(train['Image']):#using tqdm we can get a progress bar showing how much its done
    final_path=os.path.join(path,i) #setting up path
    img=cv2.imread(final_path) #reading the image
    img=cv2.resize(img,(img_h,img_w)) #resizing the image
    img=img.astype('float32') #Converting all the pixels as float32
    train_img.append(img) #Append it to the list.
    train_label.append(train['target'][j]) #similarly append the label
    j=j+1

100%|██████████| 364/364 [00:02<00:00, 138.05it/s]


In [ ]:
test_img=[]
path='/content/drive/My Drive/dataset/test'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(img_h,img_w))
    img=img.astype('float32')
    test_img.append(img)

100%|██████████| 156/156 [00:01<00:00, 132.55it/s]


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid=train_test_split(train_img,train_label,test_size=0.05,shuffle=True)

In [ ]:
'''Rescale is a value by which we will multiply the data before any other processing. 
Our original images consist in RGB coefficients in the 0-255, but such values would be too high for our model to process 
(given a typical learning rate), so we target values between 0 and 1 instead by scaling with a 1/255. factor'''
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,# divide each input by its std
        rescale=1./255,
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


In [ ]:
#We don't do any changes for the test data.
test_datagen= ImageDataGenerator(rescale=1./255)
valid_datagen= ImageDataGenerator(rescale=1./255)
train_datagen.fit(x_train)
test_datagen.fit(test_img)
valid_datagen.fit(x_valid)

In [ ]:
train_img=np.array(train_img)
x_train= np.array(x_train)
x_valid= np.array(x_valid)
y_train= np.array(y_train)
y_valid= np.array(y_valid)
test_img=np.array(test_img)
train_label=np.array(train_label)
print("Shape of training data=",x_train.shape," and shape of labels of training data= ",y_train.shape)
print("Shape of validation data=",x_valid.shape," and shape of labels of validation data= ",y_valid.shape)
print("Shape of test data=",test_img.shape)

Shape of training data= (345, 224, 224, 3)  and shape of labels of training data=  (345,)
Shape of validation data= (19, 224, 224, 3)  and shape of labels of validation data=  (19,)
Shape of test data= (156, 224, 224, 3)


In [ ]:
'''Flatten -> is the function that converts the pooled feature map to a single column that is passed to the fully connected layer.
Dense -> adds the fully connected layer to the neural network.
Dropout->regularization
BatchNormalization-> sends data(Images) as the batches

Model -> Model groups layers into an object with training and inference features.
Sequential -> Sequential groups a linear stack of layers into a tf.keras.Model.

to_categorical -> Converts a class vector (integers) to binary class matrix.

Conv2D -> This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs
MaxPooling2D -> Downsamples the input representation by taking the maximum value over the window defined by pool_size for each dimension along the features axis

ReduceLROnPlateau -> Reduce learning rate when a metric(accuracy) has stopped improving.
'''
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)
callbacks = [reduce_learning_rate]

### VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
base_model=VGG16(include_top=False,weights='imagenet',input_shape=(img_h,img_w,3),pooling='max')

In [ ]:
for layer in base_model.layers[:-4]:
    layer.trainable=False

In [ ]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dense(8,activation='sigmoid'))
model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# vgg19-3
model=Sequential()
model.add(base_model)
model.add(Flatten())
#model_3.add(BatchNormalization())
#model_3.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
#model_3.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
#model_3.add(BatchNormalization())
model.add(Dense(8,activation='softmax'))

model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#model_3.summary()

In [ ]:
model.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=16),epochs=200,callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=16),verbose=1)

Epoch 1/200
22/22 [==============================] - 14s 658ms/step - loss: 2.3353 - accuracy: 0.1797 - val_loss: 1.8624 - val_accuracy: 0.2632 - lr: 0.0010
Epoch 2/200
22/22 [==============================] - 14s 638ms/step - loss: 1.7499 - accuracy: 0.2957 - val_loss: 1.3675 - val_accuracy: 0.5789 - lr: 0.0010
Epoch 3/200
22/22 [==============================] - 14s 635ms/step - loss: 1.6098 - accuracy: 0.3681 - val_loss: 1.2788 - val_accuracy: 0.4737 - lr: 0.0010
Epoch 4/200
22/22 [==============================] - 14s 635ms/step - loss: 1.5389 - accuracy: 0.4058 - val_loss: 1.0228 - val_accuracy: 0.4737 - lr: 0.0010
Epoch 5/200
22/22 [==============================] - 14s 645ms/step - loss: 1.4248 - accuracy: 0.4493 - val_loss: 1.3055 - val_accuracy: 0.4737 - lr: 0.0010
Epoch 6/200
22/22 [==============================] - 14s 646ms/step - loss: 1.1962 - accuracy: 0.5362 - val_loss: 1.6331 - val_accuracy: 0.5789 - lr: 0.0010
Epoch 7/200
22/22 [==============================] - 14s 6

In [ ]:
model.evaluate(train_datagen.flow(x_train, to_categorical(y_train,8))) #epochs=200

11/11 [==============================] - 3s 287ms/step - loss: 0.0446 - accuracy: 0.9826


[0.044570181518793106, 0.9826086759567261]

In [ ]:
model.evaluate(train_datagen.flow(x_train, to_categorical(y_train,8))) #epochs=100

11/11 [==============================] - 3s 290ms/step - loss: 0.0896 - accuracy: 0.9768


[0.08958088606595993, 0.9768115878105164]

In [ ]:
model.evaluate(train_datagen.flow(x_train, to_categorical(y_train,8))) #epochs=30

11/11 [==============================] - 3s 287ms/step - loss: 0.1789 - accuracy: 0.9391


[0.1789371520280838, 0.939130425453186]

In [ ]:
l1= model.predict(test_img)

In [ ]:
labels = model.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head()

,Image,target
0,508.jpg,odissi
1,246.jpg,odissi
2,473.jpg,odissi
3,485.jpg,odissi
4,128.jpg,kuchipudi


In [ ]:
submission['target'].value_counts()

odissi           73
kathakali        27
mohiniyattam     16
manipuri         15
kuchipudi        10
bharatanatyam     6
sattriya          6
kathak            3
Name: target, dtype: int64

In [ ]:
labels = model.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head()

,Image,target
0,508.jpg,kuchipudi
1,246.jpg,kuchipudi
2,473.jpg,odissi
3,485.jpg,odissi
4,128.jpg,kuchipudi


In [ ]:
submission.to_csv('/content/drive/My Drive/dataset/vgg16.csv', index=False)

## VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
base_model_3=VGG19(include_top=False, weights='imagenet',input_shape=(img_h,img_w,3), pooling='max')

80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model_3.layers[:-4]:
    layer.trainable=False

In [ ]:
# vgg19-4
model_3=Sequential()
model_3.add(base_model_3)
model_3.add(Flatten())
#model_3.add(BatchNormalization())
#model_3.add(Dropout(0.2))

model_3.add(Dense(512, activation='relu'))
#model_3.add(BatchNormalization())
model_3.add(Dropout(0.2))

model_3.add(Dense(256, activation='relu'))
model_3.add(Dropout(0.2))
model_3.add(Dense(128, activation='relu'))
#model_3.add(BatchNormalization())
model_3.add(Dense(8,activation='softmax'))

model_3.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#model_3.summary()

In [ ]:
# vgg19-3
model_3=Sequential()
model_3.add(base_model_3)
model_3.add(Flatten())
#model_3.add(BatchNormalization())
#model_3.add(Dropout(0.2))
model_3.add(Dense(512, activation='relu'))
#model_3.add(BatchNormalization())
model_3.add(Dropout(0.2))
model_3.add(Dense(128, activation='relu'))
#model_3.add(BatchNormalization())
model_3.add(Dense(8,activation='softmax'))

model_3.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#model_3.summary()

In [ ]:
model_3.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=16),epochs=200,callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=16),verbose=1)

Epoch 1/200
22/22 [==============================] - 4s 167ms/step - loss: 0.2061 - accuracy: 0.9420 - val_loss: 0.3218 - val_accuracy: 0.8421 - lr: 1.0000e-05
Epoch 2/200
22/22 [==============================] - 4s 167ms/step - loss: 0.1935 - accuracy: 0.9536 - val_loss: 0.3276 - val_accuracy: 0.8421 - lr: 1.0000e-05
Epoch 3/200
22/22 [==============================] - 4s 166ms/step - loss: 0.1764 - accuracy: 0.9420 - val_loss: 0.3280 - val_accuracy: 0.8421 - lr: 1.0000e-05
Epoch 4/200
22/22 [==============================] - 4s 166ms/step - loss: 0.1879 - accuracy: 0.9391 - val_loss: 0.3316 - val_accuracy: 0.8421 - lr: 1.0000e-05
Epoch 5/200
22/22 [==============================] - 4s 166ms/step - loss: 0.1453 - accuracy: 0.9536 - val_loss: 0.3373 - val_accuracy: 0.8421 - lr: 1.0000e-05
Epoch 6/200
22/22 [==============================] - 4s 168ms/step - loss: 0.1607 - accuracy: 0.9478 - val_loss: 0.3422 - val_accuracy: 0.8421 - lr: 1.0000e-05
Epoch 7/200
22/22 [=====================

In [ ]:
model_3.evaluate(train_datagen.flow(x_train, to_categorical(y_train,8))) #epochs=200

11/11 [==============================] - 3s 291ms/step - loss: 0.0395 - accuracy: 0.9913


[0.03948166221380234, 0.991304337978363]

In [ ]:
labels = model_3.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head()

,Image,target
0,508.jpg,odissi
1,246.jpg,odissi
2,473.jpg,odissi
3,485.jpg,odissi
4,128.jpg,bharatanatyam


In [ ]:
submission['target'].value_counts()

odissi           46
bharatanatyam    31
kathakali        21
kathak           20
kuchipudi        14
mohiniyattam     13
manipuri          6
sattriya          5
Name: target, dtype: int64

In [ ]:
model_3.evaluate(train_datagen.flow(x_train, to_categorical(y_train,8))) #epochs=100

11/11 [==============================] - 3s 297ms/step - loss: 0.1603 - accuracy: 0.9594


[0.1603143811225891, 0.9594202637672424]

In [ ]:
labels = model_3.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head()

,Image,target
0,508.jpg,bharatanatyam
1,246.jpg,odissi
2,473.jpg,odissi
3,485.jpg,odissi
4,128.jpg,bharatanatyam


In [ ]:
submission['target'].value_counts()

odissi           45
bharatanatyam    34
kathakali        23
kathak           20
kuchipudi        12
mohiniyattam     12
manipuri          6
sattriya          4
Name: target, dtype: int64

In [ ]:
submission.to_csv('/content/drive/My Drive/dataset/vgg19.csv', index=False)

In [ ]:
y=labels

In [ ]:
label = [np.argmax(i) for i in y]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission['target'].value_counts()

odissi           30
kuchipudi        29
kathakali        23
kathak           23
bharatanatyam    22
mohiniyattam     13
sattriya         10
manipuri          6
Name: target, dtype: int64

In [ ]:
label = [np.argmax(i) for i in y]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission['target'].value_counts()

odissi           30
kuchipudi        29
kathakali        23
kathak           23
bharatanatyam    22
mohiniyattam     13
sattriya         10
manipuri          6
Name: target, dtype: int64

In [ ]:
submission.to_csv('/content/drive/My Drive/dataset/vgg16+19.csv', index=False)

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate
def stacking_ensemble(members,input_shape,n_classes):
  commonInput = Input(shape=input_shape)
  out=[]

  for model in members:
    #model._name= model._name+"test"+ str(members.index(model)+1)
    model._name= model.get_layer(index = 0)._name +"-test"+ str(members.index(model)+1)
    out.append(model(commonInput))

  modeltmp = concatenate(out,axis=-1)
  modeltmp = Dense(32, activation='relu')(modeltmp)
  modeltmp = Dense(16, activation='relu')(modeltmp)
  modeltmp = Dense(n_classes, activation='softmax')(modeltmp)
  stacked_model = Model(commonInput,modeltmp)
  stacked_model.compile( loss='categorical_crossentropy',optimizer= 'adam', metrics=['accuracy'])

  return stacked_model

In [ ]:
members=[model,model_3]

In [ ]:
stacked_model= stacking_ensemble(members,(img_h,img_w,3),8)
stacked_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg16-test1 (Sequential)        (None, 8)            15044040    input_3[0][0]                    
__________________________________________________________________________________________________
vgg19-test2 (Sequential)        (None, 8)            20353736    input_3[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 16)           0           vgg16-test1[1][0]                
                                                                 vgg19-test2[1][0]            

In [ ]:
stacked_model.fit(train_datagen.flow(x_train, to_categorical(y_train,8), batch_size=32),epochs=200,callbacks=callbacks,
          validation_data= valid_datagen.flow(x_valid, to_categorical(y_valid,8), batch_size=32),verbose=1)

11/11 [==============================] - 4s 343ms/step - loss: 1.3793 - accuracy: 0.5130 - val_loss: 1.3525 - val_accuracy: 0.4737 - lr: 1.0000e-05
Epoch 49/200
10/11 [==========================>...] - ETA: 0s - loss: 1.3810 - accuracy: 0.5112

KeyboardInterrupt: ignored

In [ ]:
model_3.evaluate(train_datagen.flow(x_train, to_categorical(y_train,8))) #epochs=100

In [ ]:
labels = stacked_model.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.head(10)

In [ ]:
 submission.to_csv('/content/drive/My Drive/dataset/stacking.csv', index=False)